In [21]:
import pandas as pd
import numpy as np

print('import fin')

import fin


# base dataframe by datetime

In [22]:
# start and end time of data
time_start = pd.to_datetime('2018-04-01 00:00:00')
time_end = pd.to_datetime('2021-01-31 23:00:00')

# how many time tags
time_len = int((time_end - time_start) / pd.Timedelta(hours=1)) + 1

# Make base dataframe
delta = pd.to_timedelta(np.arange(time_len), unit='h')
time_base = time_start + delta

# set base
data_base = pd.DataFrame()
data_base['date'] = time_base
data_base['time_idx'] = data_base.index

data_base['month'] = data_base['date'].dt.month_name() # string
data_base['week_of_year'] = data_base['date'].dt.week
data_base['day_of_month'] = data_base['date'].dt.day.apply(str)
data_base['hour_of_day'] = data_base['date'].dt.hour.apply(str)

# --------------------
print(f'Data starts from [ {time_start} ] to [ {time_end} ].')
print(f'Total length : {time_len}')


Data starts from [ 2018-04-01 00:00:00 ] to [ 2021-01-31 23:00:00 ].
Total length : 24888


<ipython-input-22-f563864df158>:18: FutureWarning: Series.dt.weekofyear and Series.dt.week have been deprecated.  Please use Series.dt.isocalendar().week instead.
  data_base['week_of_year'] = data_base['date'].dt.week


# functions

In [23]:
def convert2400(time_str):
    '''
    convert type of time data, string -> datetime
    e.g. < string "2020-01-01 24:00:00" > to < datetime "2020-01-02 00:00:00" >
    '''
    if "24:00:00" in time_str:
        tmp = time_str.replace("24:00:00", "00:00:00")
        tmp = pd.to_datetime(tmp)
        tmp = tmp + pd.Timedelta(days=1)
    else:
        tmp = pd.to_datetime(time_str)
    return tmp


def new_obs_data(region, data_path):
    '''
    get fcst data.
    '''
    # load csv file
    df_obs = pd.read_csv(f'{data_path}/new_obs_{region}.csv')

    # create empty dataframe
    data_obs = pd.DataFrame(columns=['date', 'temperature', 'humidity', 
                                   'wind_speed', 'wind_direct', 'cloud'])

    # wind_direction simplification
    bin_boundary = list(range(0, 361, 45))
    bin_names = ['NNE', 'ENE', 'ESE', 'SSE', 'SSW', 'WSW', 'WNW', 'NNW']
    wd = pd.cut(df_obs['풍향(16방위)'],
              bins=bin_boundary,
              labels=bin_names,
              include_lowest=True)

    # fill dataframe
    data_obs['date'] = df_obs['일시'].apply(convert2400)
    data_obs['temperature'] = df_obs['기온(°C)']
    data_obs['wind_speed'] = df_obs['풍속(m/s)']
    data_obs['wind_direct'] = wd
    data_obs['humidity'] = df_obs['습도(%)']
    data_obs['cloud'] = df_obs['전운량(10분위)']
    
    data_obs['cloud_low'] = df_obs['중하층운량(10분위)']
    data_obs['rain'] = df_obs['강수량(mm)']
    data_obs['vapor_p'] = df_obs['증기압(hPa)']
    data_obs['sea_p'] = df_obs['해면기압(hPa)']
    data_obs['sunshine'] = df_obs['일조(hr)']

    return data_obs


def get_energy_data(data_path):
    '''
    get energy data.
    '''
    # load csv file
    df_energy = pd.read_csv(f'{data_path}/energy.csv')

    # create empty dataframe
    data_energy = pd.DataFrame(columns=['date', 'ulsan', 'dj_warehouse',
                                      'dj_floating', 'dj'])

    # fill dataframe
    data_energy['date'] = df_energy['time'].apply(convert2400) - pd.Timedelta(hours=1)
    data_energy['ulsan'] = df_energy['ulsan']
    data_energy['dj_warehouse'] = df_energy['dangjin_warehouse']
    data_energy['dj_floating'] = df_energy['dangjin_floating']
    data_energy['dj'] = df_energy['dangjin']

    return data_energy

def get_plant_energy(data_energy, plant):
    '''
    get only selected plant energy data
    '''
    # dictionary for region of plant
    dict_region = {'ulsan': 'ulsan',
                 'dj_warehouse': 'dangjin',
                 'dj_floating': 'dangjin',
                 'dj': 'dangjin'}

    # create empty dataframe
    data_plant_energy = pd.DataFrame(columns=['date', 'energy', 'region', 'plant'])

    # choose only selected plant energy
    data_plant_energy['date'] = data_energy['date']
    data_plant_energy['energy'] = data_energy[plant]
    data_plant_energy['region'] = dict_region[plant]
    data_plant_energy['plant'] = plant

    return data_plant_energy


def process_data(data, d_type):
    '''
    fcst, obs data
    interpolation, fill nan values
    '''

    # copy origin dataframe
    data_filled = data.copy()

    # drop duplicates
    data_filled.drop_duplicates(subset=['date'], keep='last', inplace=True)
    data_filled.reset_index(drop=True, inplace=True)

    # split columns : real var or categorical var
    col_real_linear = ['date', 'temperature', 'humidity', 'wind_speed', 'vapor_p', 'sea_p']
    col_real_zero = ['cloud', 'cloud_low', 'rain', 'sunshine']
    col_cat = ['wind_direct']

    # interpolation
    real_linear = data_filled[col_real_linear].copy()
    real_zero = data_filled[col_real_zero].copy()
    cat = data_filled[col_cat].copy()

    real_linear = real_linear.set_index('date')
    real_linear = real_linear.interpolate(method='time') # Issue I1
    real_linear = real_linear.reset_index()
    real_zero.fillna(0, inplace=True) # NaN -> no cloud, rain, sunshine
    cat = cat.interpolate(method='ffill')

    # apply interpolation
    data_filled[col_real_linear] = real_linear
    data_filled[col_real_zero] = real_zero
    data_filled[col_cat] = cat

    return data_filled

def process_energy(data):
    '''
    energy data
    interpolation, fill nan values
    '''
    # copy origin dataframe
    data_filled = data.copy()

    # drop duplicates
    data_filled.drop_duplicates(subset=['date'], keep='last', inplace=True)
    data_filled.reset_index(inplace=True)

    # interpolation
    data_filled = data_filled.set_index('date')
    data_filled = data_filled.interpolate(method='time')
    data_filled = data_filled.reset_index()

    return data_filled

# Get dataset and Make origin data

In [24]:
"""
< 총 column 개수는 19개 >

+ 시간 관련 6개(time, idx, month, week_of_year, day_of_month, hour_of_day)
+ fcst 5개(기온, 습도, 풍향, 풍속, 구름)
+ obs 5개(기온, 습도, 풍향, 풍속, 구름)
+ energy 1개(energy)
+ 지역정보 2개(region, plant)
"""

# Default Params
data_path = './dataset'
data_path_new = './new_dataset'
region_default = {'dangjin_floating': 'dangjin',
                  'dangjin_warehouse': 'dangjin',
                  'dangjin': 'dangjin',
                  'ulsan': 'ulsan'} # plant, region

# dataframes for use
data = pd.DataFrame()
data_obs_ulsan = new_obs_data('ulsan', data_path_new)
data_obs_dangjin = new_obs_data('dangjin', data_path_new)
data_energy = get_energy_data(data_path)

data_obs_ulsan = pd.merge(data_base, data_obs_ulsan, how='left', on='date')
data_obs_dangjin = pd.merge(data_base, data_obs_dangjin, how='left', on='date')
data_energy = pd.merge(data_base, data_energy, how='left', on='date')

# dataframe에 결측치를 사전에 선형 보간하기
data_obs_dangjin = process_data(data_obs_dangjin, 'obs')
data_obs_ulsan = process_data(data_obs_ulsan, 'obs')
data_energy = process_energy(data_energy)

# 출력된 energy에 따라서 plant별로 Dataframe 만들기
data_ulsan = pd.DataFrame()         # ulsan
data_dj_warehouse = pd.DataFrame()  # dangjin_warehouse
data_dj_floating = pd.DataFrame()   # dangjin_floating
data_dj = pd.DataFrame()            # dangjin

col_time = data_base.columns.to_list()  # 공통으로 가진 column

data_ulsan = pd.merge(data_base, data_obs_ulsan, how='left', on=col_time)
data_ulsan = pd.merge(data_ulsan, get_plant_energy(data_energy, 'ulsan'),
                      how='left', on='date')

data_dj_warehouse = pd.merge(data_base, data_obs_dangjin, how='left', on=col_time)
data_dj_warehouse = pd.merge(data_dj_warehouse, get_plant_energy(data_energy, 'dj_warehouse'),
                             how='left', on='date')

data_dj_floating = pd.merge(data_base, data_obs_dangjin, how='left', on=col_time)
data_dj_floating = pd.merge(data_dj_floating, get_plant_energy(data_energy, 'dj_floating'),
                            how='left', on='date')

data_dj = pd.merge(data_base, data_obs_dangjin, how='left', on=col_time)
data_dj = pd.merge(data_dj, get_plant_energy(data_energy, 'dj'),
                   how='left', on='date')

# 모든 data를 합친 Dataframe도 만들기
data = pd.concat([data_ulsan, data_dj_warehouse, data_dj_floating, data_dj])

# 일단 ulsan data만 해보기
data = data_ulsan

In [25]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 24888 entries, 0 to 24887
Data columns (total 19 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   date          24888 non-null  datetime64[ns]
 1   time_idx      24888 non-null  int64         
 2   month         24888 non-null  object        
 3   week_of_year  24888 non-null  int64         
 4   day_of_month  24888 non-null  object        
 5   hour_of_day   24888 non-null  object        
 6   temperature   24888 non-null  float64       
 7   humidity      24888 non-null  float64       
 8   wind_speed    24888 non-null  float64       
 9   wind_direct   24888 non-null  category      
 10  cloud         24888 non-null  float64       
 11  cloud_low     24888 non-null  float64       
 12  rain          24888 non-null  float64       
 13  vapor_p       24888 non-null  float64       
 14  sea_p         24888 non-null  float64       
 15  sunshine      24888 non-null  float6

In [26]:
data.isnull().sum()

date            0
time_idx        0
month           0
week_of_year    0
day_of_month    0
hour_of_day     0
temperature     0
humidity        0
wind_speed      0
wind_direct     0
cloud           0
cloud_low       0
rain            0
vapor_p         0
sea_p           0
sunshine        0
energy          0
region          0
plant           0
dtype: int64

In [28]:
data_dj.isnull().sum()

date            0
time_idx        0
month           0
week_of_year    0
day_of_month    0
hour_of_day     0
temperature     0
humidity        0
wind_speed      0
wind_direct     0
cloud           0
cloud_low       0
rain            0
vapor_p         0
sea_p           0
sunshine        0
energy          0
region          0
plant           0
dtype: int64